# 1. Download and open, mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/Colab Notebooks/Project1

/content/drive/MyDrive/Colab Notebooks/Project1


# 2. Import all lib


Đầu tiên chúng ta cần clone mã nguồn của OpenNMT về và tải các yêu cầu của nó giúp cho nó hoạt động

In [1]:
!git clone https://github.com/OpenNMT/OpenNMT-py

UsageError: Line magic function `%git` not found.


In [5]:
!pip install OpenNMT-py

     |████████████████████████████████| 216 kB 30.3 MB/s 
     |████████████████████████████████| 73 kB 2.4 MB/s 
     |████████████████████████████████| 16.3 MB 32.9 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 1.2 MB 66.0 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


# 3. Tiền xử lý dữ liệu

## Tách bộ dữ liệu thành các tập tương ứng
Bộ dữ liệu dạng text gồm nhiều dòng, mỗi dòng gồm một câu Tiếng Việt và bản dịch tiếng Khmer tương ứng được phân cách nhau bởi dấu tab. Bộ dữ liệu có tổng cộng 11917 dòng tuy nhiên một số dòng bị lỗi có thể chỉ có một câu tiếng Việt hoặc một câu tiếng Khmer hoặc không phân cách bằng kí tự tab,... Số lượng dòng lỗi như vậy khoảng 2000 dòng, tức chiếm khoảng 1.6\% bộ dữ liệu gốc, số lượng không quá to so với bộ dữ liệu.

In [2]:
str = 'data/dữ liệu song ngữ Việt - Khmer.txt'

vn_kmer_txt = open(str, mode='r', encoding='utf8')
src_train = open('./data/src-train.txt', mode='w+', encoding='utf8')
tgt_train = open('./data/tgt-train.txt', mode='w+', encoding='utf8')
src_val = open('./data/src-val.txt', mode='w+', encoding='utf8')
tgt_val = open('./data/tgt-val.txt', mode='w+', encoding='utf8')
src_test = open('./data/src-test.txt', mode='w+', encoding='utf8')
tgt_test = open('./data/tgt-test.txt', mode='w+', encoding='utf8')

if (vn_kmer_txt != None):
    print('Mở thành công file dữ liệu')

l = 0
n = 0

for line in vn_kmer_txt:
    data = line.split('\t')
    if (len(data) < 2):
        l += 1
        continue
    if (n < 5000):
        src_val.write(data[-1])
        tgt_val.write(data[0] + '\n')
    elif (n < 110000):
        src_train.write(data[-1])
        tgt_train.write(data[0]  + '\n')
    else:
        src_test.write(data[-1])
        tgt_test.write(data[0] + '\n')
    l += 1
    n += 1

print('Số lượng dòng tổng cộng là ', l)
print('Số lượng dòng đúng quy tắc có thể tách là ', n)

Mở thành công file dữ liệu
Số lượng dòng tổng cộng là  119117
Số lượng dòng đúng quy tắc có thể tách là  117180


## Tách từ đối với từng ngôn ngữ

### Tách từ đối với ngôn ngữ nguồn tiếng Khmer
Sử dụng thư viện khmer-nltk để tách các từ trong tiếng Khmer

In [ ]:
!pip install khmer-nltk

In [3]:
from khmernltk import word_tokenize

src_train = open('./data/src-train.txt', mode='r', encoding='utf8')
src_val = open('./data/src-val.txt', mode='r', encoding='utf8')
src_test = open('./data/src-test.txt', mode='r', encoding='utf8')

src_train_token = open('./data/src-train-token.txt', mode='w+', encoding='utf8')
src_val_token = open('./data/src-val-token.txt', mode='w+', encoding='utf8')
src_test_token = open('./data/src-test-token.txt', mode='w+', encoding='utf8')

n = 0;
num_word = 0
for line in src_test:
    src_test_token.write(word_tokenize(line, return_tokens=False) + "\n")
    n+=1
print('Số lượng câu trong tập huấn luyện nguồn là ', n)

n = 0;
for line in src_train:
    src_train_token.write(word_tokenize(line, return_tokens=False) + "\n")
    n+=1
print('Số lượng câu trong tập huấn luyện nguồn là ', n)

n = 0;
for line in src_val:
    src_val_token.write(word_tokenize(line, return_tokens=False) + "\n")
    n+=1
print('Số lượng câu trong tập huấn luyện nguồn là ', n)

print("Tokenize sucess")

| 2022-02-22 20:01:28,470 | INFO | khmer-nltk | Loaded model from C:\Users\Admin\anaconda3\envs\projectdata\lib\site-packages\khmernltk\word_tokenize\sklearn_crf_ner_10000.sav |
| 2022-02-22 20:01:28,482 | INFO | khmer-nltk | Loaded model from C:\Users\Admin\anaconda3\envs\projectdata\lib\site-packages\khmernltk\pos_tag\sklearn_crf_pos_alt_0.9846.sav |


Số lượng câu trong tập huấn luyện nguồn là  7180
Số lượng câu trong tập huấn luyện nguồn là  105000
Số lượng câu trong tập huấn luyện nguồn là  5000
Tokenize sucess


### Tách từ đối với ngôn ngữ nguồn tiếng Việt
Sử dụng thư viện mã nguồn mở underthesea với model word_tokenize để tách các từ trong tiếng Việt

In [ ]:
!pip install underthesea

In [4]:
from underthesea import word_tokenize

tgt_train = open('./data/tgt-train.txt', mode='r', encoding='utf8')
tgt_val = open('./data/tgt-val.txt', mode='r', encoding='utf8')
tgt_test = open('./data/tgt-test.txt', mode='r', encoding='utf8')

tgt_train_token = open('./data/tgt-train-token.txt', mode='w+', encoding='utf8')
tgt_val_token = open('./data/tgt-val-token.txt', mode='w+', encoding='utf8')
tgt_test_token = open('./data/tgt-test-token.txt', mode='w+', encoding='utf8')

n = 0;
for line in tgt_train:
    tgt_train_token.write(word_tokenize(line, format="text") + "\n")
    n+=1
print('Số lượng câu trong tập huấn luyện đích là ', n)
n = 0
for line in tgt_val:
    tgt_val_token.write(word_tokenize(line, format="text") + "\n")
    n+=1
print('Số lượng câu trong tập thẩm định đích là ', n)
n = 0
for line in tgt_test:
    tgt_test_token.write(word_tokenize(line, format="text") + "\n")
    n+=1
print('Số lượng câu trong tập kiểm tra đích là ', n)
print("Tokenize sucess")

Số lượng câu trong tập huấn luyện đích là  105000
Số lượng câu trong tập thẩm định đích là  5000
Số lượng câu trong tập kiểm tra đích là  7180
Tokenize sucess


## Mã hóa BPE cho từng tập

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i ./data/src-train-token.txt -o ./data/src.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i ./data/tgt-train-token.txt -o ./data/tgt.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/src.code -i ./data/src-train-token.txt -o ./data/src-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/src.code -i ./data/src-val-token.txt -o ./data/src-val-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/tgt.code -i ./data/src-test-token.txt -o OpenNMT-py/data/src-test-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/tgt.code -i ./data/tgt-train-token.txt -o ./data/tgt-train-bpe.txt

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c ./data/tgt.code -i ./data/tgt-val-token.txt -o ./data/tgt-val-bpe.txt

# 4. Huấn luyện mô hình
OpenNMT cũng cung cấp nhiều kiến trúc mô hình khác nhau và nhiều tùy chọn khác để tăng hiệu quả của mô hình.

Bộ mã hóa và giải mã là tổng hợp xếp chồng lên nhau của 6 layer. Mỗi layer bao gồm 2 layer con (sub-layer) trong đó sub-layer đầu tiên là multi- head self- attention với số head là 16. Sub-layer thứ hai là feedforward network. Đầu ra của mỗi sub-layer này sẽ có số chiều là 1024.

Đầu tiên, chúng ta sẽ kiểm tra xem có GPU cấp phát không? Nếu không được

In [3]:
!nvidia-smi

Mon Feb 21 01:08:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Huấn luyện mô hình với mã hóa BPE trước

In [ ]:
!onmt_build_vocab -config khmer-viet.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-02-20 03:45:57,649 INFO] Counter vocab from 10000 samples.
[2022-02-20 03:45:57,649 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-02-20 03:45:57,663 INFO] corpus_1's transforms: TransformPipe()
[2022-02-20 03:45:58,787 WARNING] Empty line exists in corpus_1#8170.
[2022-02-20 03:45:58,972 INFO] Counters src:8380
[2022-02-20 03:45:58,972 INFO] Counters tgt:8343


In [ ]:
!python OpenNMT-py/train.py -config khmer-viet.yaml -train_from data/run/model_step_20000.pt

[2022-02-20 02:34:31,563 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-02-20 02:34:31,564 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-02-20 02:34:31,564 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-20 02:34:31,565 INFO] Parsed 2 corpora from -data.
[2022-02-20 02:34:31,565 INFO] Loading checkpoint from data/run/model_step_20000.pt
[2022-02-20 02:34:32,055 INFO] Loading fields from checkpoint...
[2022-02-20 02:34:32,055 INFO]  * src vocab size = 8382
[2022-02-20 02:34:32,055 INFO]  * tgt vocab size = 8347
[2022-02-20 02:34:32,058 INFO] Building model...
[2022-02-20 02:34:35,094 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8382, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.3, inplace=False)
        )
      )
  

## Huấn luyện mô hình không có mã hóa BPE

Tại đây, em sử dụng kĩ thuật học chuyển tiếp. Sử dụng cùng mô hình dịch như trên và sử dụng những tham số đã được học từ mô hình trên sau 20000 bước để tiếp tục huấn luyện cho mô hình này.

In [ ]:
!onmt_build_vocab -config khmer-viet-no-bpe.yaml -n_sample 10000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-02-20 17:09:25,736 INFO] Counter vocab from 10000 samples.
[2022-02-20 17:09:25,736 INFO] Build vocab on 10000 transformed examples/corpus.
[2022-02-20 17:09:25,750 INFO] corpus_1's transforms: TransformPipe()
[2022-02-20 17:09:28,288 WARNING] Empty line exists in corpus_1#8170.
[2022-02-20 17:09:28,463 INFO] Counters src:12813
[2022-02-20 17:09:28,463 INFO] Counters tgt:11829


In [6]:
!python OpenNMT-py/train.py -config khmer-viet-no-bpe.yaml -train_from data/run2/model_step_1000.pt

[2022-02-21 01:10:44,247 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2022-02-21 01:10:44,247 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2022-02-21 01:10:44,247 INFO] Missing transforms field for valid data, set to default: [].
[2022-02-21 01:10:44,248 INFO] Parsed 2 corpora from -data.
[2022-02-21 01:10:44,248 INFO] Loading checkpoint from data/run2/model_step_1000.pt
[2022-02-21 01:10:50,682 INFO] Loading fields from checkpoint...
[2022-02-21 01:10:50,682 INFO]  * src vocab size = 12815
[2022-02-21 01:10:50,682 INFO]  * tgt vocab size = 11833
[2022-02-21 01:10:50,688 INFO] Building model...
[2022-02-21 01:11:00,681 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(12815, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.3, inplace=False)
        )
      )

# 5. Dịch
Sau khi huấn luyện một mô hình, chúng ta sẽ có thể sử dụng mô hình đã được huấn luyện ở file model\_step\_XXX.pt. Mô hình được huấn luyện chiếm ít dung lượng bộ nhớ và tương thích với CPU và GPU. Nó có thể dùng để dịch.

Theo mặc định, OpenNMT sẽ tạo ra bản dịch bằng cách sử dụng tìm kiếm chùm với beam\_size có thể tùy chọn nhằm lựa chọn thời gian và độ chính xác tìm kiếm, với beam\_size bằng 1 cho phép tìm kiếm tham lam.

## Dịch với mô hình sử dụng mã hóa BPE

In [ ]:
!python OpenNMT-py/translate.py -model data/run/model_step_30000.pt -src data/src-test-token.txt -output data/pred_bpe.txt -verbose -gpu 0

Streaming output truncated to the last 5000 lines.
SENT 6181: ['ដូច្នេះ', 'វិស័យ', 'អប់រំ', 'កំណត់', 'នេះ', 'ថា', 'ជា', 'អត្តសញ្ញាណ', 'ជា', 'ទំនួល', 'ខុសត្រូវ', 'និង', 'បេសកកម្ម', 'របស់', 'ឧស្សាហកម្ម', 'និង', 'គ្រូ', '។']
PRED 6181: Vì_vậy , ngành giáo_dục xác_định đây là bản_sắc , là trách_nhiệm và nghĩa_vụ của ngành công_nghiệp , giáo_viên .
PRED SCORE: -5.2400

[2022-02-20 16:54:37,698 INFO] 
SENT 6182: ['តែ', 'តាមពិត', 'នេះ', 'ជា', 'ការបង្ក', 'គ្រោះថ្នាក់', 'ដោយ', 'ចៃដន្យ', 'ហើយ', 'ធ្វើឱ្យ', 'កុមារ', 'ក្លាយជា', 'ស្ត្រេស', 'ធុញទ្រាន់', 'និង', 'ស្អប់', 'នឹង', 'ការសិក្សា', '។']
PRED 6182: Trên thực_tế , đây là những kẻ nguy_hiểm , gây tình_@@ cờ và tình_@@ cờ cho trẻ_em mầm_non và cô gái .
PRED SCORE: -11.3240

[2022-02-20 16:54:37,699 INFO] 
SENT 6183: ['នៅលើ', 'ទីផ្សារ', 'មាស', 'ពិភពលោក', 'តម្លៃ', 'មាស', 'នៅ', 'សហរដ្ឋ', 'អាមេរិក', 'បាន', 'កើនឡើង', '៧,៣', 'ដុល្លារ', 'អាមេរិក', 'ដល់', '១,៨៧២,១', 'ដុល្លារ', '/', 'អោន', '។']
PRED 6183: Trên thị_trường vàng thế_giới , giá vàng ở Hoa_Kỳ đ

## Dịch với mô hình không sử dụng BPE

In [8]:
!python OpenNMT-py/translate.py -model data/run2/model_step_30000.pt -src data/src-test-token.txt -output data/pred_no_bpe.txt -verbose -gpu 0

Streaming output truncated to the last 5000 lines.
SENT 6181: ['ដូច្នេះ', 'វិស័យ', 'អប់រំ', 'កំណត់', 'នេះ', 'ថា', 'ជា', 'អត្តសញ្ញាណ', 'ជា', 'ទំនួល', 'ខុសត្រូវ', 'និង', 'បេសកកម្ម', 'របស់', 'ឧស្សាហកម្ម', 'និង', 'គ្រូ', '។']
PRED 6181: Do_đó , lĩnh_vực giáo_dục quy_định đây là một bản_sắc trách_nhiệm và trách_nhiệm của ngành công_nghiệp và giáo_viên .
PRED SCORE: -7.9890

[2022-02-21 04:24:27,309 INFO] 
SENT 6182: ['តែ', 'តាមពិត', 'នេះ', 'ជា', 'ការបង្ក', 'គ្រោះថ្នាក់', 'ដោយ', 'ចៃដន្យ', 'ហើយ', 'ធ្វើឱ្យ', 'កុមារ', 'ក្លាយជា', 'ស្ត្រេស', 'ធុញទ្រាន់', 'និង', 'ស្អប់', 'នឹង', 'ការសិក្សា', '។']
PRED 6182: Trên thực_tế , đây là một hành_động gây nguy_hiểm , biến lời_nói thành trẻ_em mệt_mỏi và học .
PRED SCORE: -12.5631

[2022-02-21 04:24:27,309 INFO] 
SENT 6183: ['នៅលើ', 'ទីផ្សារ', 'មាស', 'ពិភពលោក', 'តម្លៃ', 'មាស', 'នៅ', 'សហរដ្ឋ', 'អាមេរិក', 'បាន', 'កើនឡើង', '៧,៣', 'ដុល្លារ', 'អាមេរិក', 'ដល់', '១,៨៧២,១', 'ដុល្លារ', '/', 'អោន', '។']
PRED 6183: Trên thị_trường vàng thế_giới , giá vàng ở Mỹ tăng lên

#6. Đánh giá bản dịch

## Đối với bản dịch sử dụng mã hóa BPE
Do bản dịch sau khi được dịch, nó vẫn là những mảnh rời rạc do mã hóa BPE nên không thể hiểu với người và so sánh với bản dịch tham chiếu tới. Do đó, chúng ta phải loại bỏ chúng và đưa về bản dịch theo như thực tế

Chúng ta có thể sử dụng lệnh !sed của linux. Lệnh sed là lệnh hiệu quả tích hợp sẵn trên linux để xử lý luồng kí tự

In [6]:
!sed 's/\_/ /g' -i  data/pred_bpe.txt 

In [7]:
!sed "s/@@ //g" -i data/pred_bpe.txt

In [ ]:
!perl OpenNMT-py/tools/multi-bleu-detok.perl ./data/tgt-test.txt < ./data/pred_bpe.txt

BLEU = 25.31, 54.5/31.9/19.3/12.2 (BP=1.000, ratio=1.029, hyp_len=200902, ref_len=195271)


## Đối với bản dịch không dùng mã hóa BPE

In [9]:
!sed 's/\_/ /g' -i  data/pred_no_bpe.txt 

In [10]:
!perl OpenNMT-py/tools/multi-bleu-detok.perl ./data/tgt-test.txt < ./data/pred_no_bpe.txt

BLEU = 24.58, 54.9/31.5/18.5/11.4 (BP=1.000, ratio=1.011, hyp_len=197324, ref_len=195271)
